<h1 style="color: #6cb4e4;  text-align: center;  padding: 0.25em;  border-top: solid 2.5px #6cb4e4;  border-bottom: solid 2.5px #6cb4e4;  background: -webkit-repeating-linear-gradient(-45deg, #f0f8ff, #f0f8ff 3px,#e9f4ff 3px, #e9f4ff 7px);  background: repeating-linear-gradient(-45deg, #f0f8ff, #f0f8ff 3px,#e9f4ff 3px, #e9f4ff 7px);height:45px;">
<b>
《《《Submission1(nfnet)》》》
</b></h1> 

In [1]:
import numpy as np  # 导入数值计算库
import torchaudio.transforms as AT  # 导入音频转换模块
import concurrent.futures  # 导入并行处理模块
import os  # 导入操作系统接口模块
import gc  # 导入垃圾收集模块
import warnings  # 导入警告控制模块
import logging  # 导入日志模块
import time  # 导入时间相关模块
import pandas as pd  # 导入数据处理库
import soundfile as sf  # 导入音频文件读写模块
import torch  # 导入PyTorch深度学习框架
import torch.nn as nn  # 导入神经网络模块
import torch.nn.functional as F  # 导入函数式接口
import timm  # 导入预训练模型库
from glob import glob  # 导入文件路径匹配模块
import torchaudio  # 导入音频处理库
import random  # 导入随机数模块
import itertools  # 导入迭代器工具
import concurrent.futures  # 导入并行处理模块

warnings.filterwarnings("ignore")  # 忽略警告信息
logging.basicConfig(level=logging.ERROR)  # 设置日志级别为ERROR

# 🐦 BirdCLEF 2025: 加权混合推理 (LB 0.878)

## 🧠 概述

本notebook使用**加权混合**方法在公共排行榜上获得了**0.878**的分数。

## 🧩 使用的模型

最终预测基于以下公共模型的集成：

- 📘 **[Bird2025 | 单一SED模型推理 [LB 0.857]](https://www.kaggle.com/code/i2nfinit3y/bird2025-single-sed-model-inference-lb-0-857)**  
  作者：[I2nfinit3y](https://www.kaggle.com/i2nfinit3y)  
  一个强大的单一SED模型，作为集成组件之一的基础。

- 🧪 **[低排名预测的幂调整后处理](https://www.kaggle.com/code/myso1987/post-processing-with-power-adjustment-for-low-rank)**  
  作者：[MYSO](https://www.kaggle.com/myso1987)  
  该notebook引入了一种巧妙的后处理技术，通过幂变换提升低置信度预测。

- 🔗 **[Bird25 | 加权混合 | nfnet + convnextv2 | LB.860](https://www.kaggle.com/code/hideyukizushi/bird25-weightedblend-nfnet-convnextv2-lb-860)**  
  作者：[yukiZ](https://www.kaggle.com/hideyukizushi)  
  提供了用于组合模型输出的核心混合逻辑。

## ⚖️ 加权混合策略

我们使用两个模型输出的加权平均值，具体如下：

- **nfnet**: 25%
- **seresnext**: 75%

虽然这种混合方法在排行榜上获得了高分，但它可能过拟合到公共测试集 — 这是2022年至2024年BirdCLEF竞赛中常见的模式。评估结果时请考虑这一点。

---

In [2]:
test_audio_dir = '../input/birdclef-2025/test_soundscapes/'  # 设置测试音频文件目录路径
file_list = [f for f in sorted(os.listdir(test_audio_dir))]  # 获取目录中所有文件的排序列表
file_list = [file.split('.')[0] for file in file_list if file.endswith('.ogg')]  # 提取所有.ogg文件的文件名（不含扩展名）

debug = False  # 初始化调试模式为关闭
if len(file_list) == 0:  # 如果测试目录为空
    debug = True  # 开启调试模式
    debug_st_num = 5  # 设置调试起始样本编号
    debug_num = 8  # 设置调试样本数量
    test_audio_dir = '../input/birdclef-2025/train_soundscapes/'  # 改用训练集音频文件作为测试
    file_list = [f for f in sorted(os.listdir(test_audio_dir))]  # 获取训练集目录中所有文件
    file_list = [file.split('.')[0] for file in file_list if file.endswith('.ogg')]  # 提取所有.ogg文件的文件名
    file_list = file_list[debug_st_num:debug_st_num+debug_num]  # 选取指定范围的文件进行调试

print('Debug mode:', debug)  # 打印当前调试模式状态
print('Number of test soundscapes:', len(file_list))  # 打印测试音频文件数量

Debug mode: True
Number of test soundscapes: 8


In [3]:
def adaptive_power(p, top_k=30, min_exp=1.5, max_exp=3.0, inplace=True):
    if not inplace:  # 如果不进行原地操作
        p = p.copy()  # 创建输入数组的副本
    col_max = p.max(axis=0)  # 计算每列的最大值
    ranks = np.argsort(-col_max)  # 根据列最大值降序排序获取索引
    exps = np.linspace(max_exp, min_exp, len(col_max))  # 创建线性递减的指数值数组
    for i, col in enumerate(ranks[top_k:]):  # 遍历排名在top_k之后的列
        p[:, col] = p[:, col] ** exps[i]  # 对每列应用不同的幂次变换
    return p  # 返回处理后的数组

In [4]:
wav_sec = 5  # 设置每个音频片段的时长为5秒
sample_rate = 32000  # 设置采样率为32kHz
min_segment = sample_rate*wav_sec  # 计算最小音频片段的样本数

class_labels = sorted(os.listdir('../input/birdclef-2025/train_audio/'))  # 获取并排序所有鸟类标签

n_fft=1024  # 设置快速傅里叶变换的窗口大小
win_length=1024  # 设置窗口长度
hop_length=512  # 设置窗口移动步长
f_min=40  # 设置最小频率
f_max=16000  # 设置最大频率
n_mels=128  # 设置梅尔频谱的频段数

mel_spectrogram = AT.MelSpectrogram(  # 创建梅尔频谱转换器
    sample_rate=sample_rate,  # 指定采样率
    n_fft=n_fft,  # 指定FFT窗口大小
    win_length=win_length,  # 指定窗口长度
    hop_length=hop_length,  # 指定窗口移动步长
    center=True,  # 启用中心填充
    f_min=f_min,  # 指定最小频率
    f_max=f_max,  # 指定最大频率
    pad_mode="reflect",  # 使用反射填充模式
    power=2.0,  # 使用功率谱
    norm='slaney',  # 使用Slaney归一化
    n_mels=n_mels,  # 指定梅尔频段数
    mel_scale="htk",  # 使用HTK梅尔尺度
)

def normalize_std(spec, eps=1e-6):  # 定义标准化函数
    mean = torch.mean(spec)  # 计算均值
    std = torch.std(spec)  # 计算标准差
    return torch.where(std == 0, spec-mean, (spec - mean) / (std+eps))  # 标准化，并处理标准差为0的情况

def audio_to_mel(filepath=None):  # 定义音频转梅尔频谱的函数
    waveform, sample_rate = torchaudio.load(filepath,backend="soundfile")  # 加载音频文件
    len_wav = waveform.shape[1]  # 获取音频长度
    waveform = waveform[0,:].reshape(1, len_wav)  # 将立体声转为单声道
    PREDS = []  # 初始化预测结果列表
    for i in range(12):  # 对每个5秒片段进行处理
        waveform2 = waveform[:,i*sample_rate*5:i*sample_rate*5+sample_rate*5]  # 提取5秒片段
        melspec = mel_spectrogram(waveform2)  # 转换为梅尔频谱
        melspec = torch.log(melspec+1e-6)  # 对频谱取对数
        melspec = normalize_std(melspec)  # 标准化频谱
        melspec = torch.unsqueeze(melspec, dim=0)  # 增加批次维度
        
        PREDS.append(melspec)  # 添加到结果列表
    return torch.vstack(PREDS)  # 垂直堆叠所有频谱并返回

In [5]:
def init_layer(layer):  # 初始化网络层函数
    nn.init.xavier_uniform_(layer.weight)  # 使用Xavier均匀分布初始化权重
    if hasattr(layer, "bias"):  # 如果层有偏置参数
        if layer.bias is not None:  # 如果偏置不为None
            layer.bias.data.fill_(0.)  # 将偏置初始化为0


def init_bn(bn):  # 初始化批归一化层函数
    bn.bias.data.fill_(0.)  # 批归一化偏置初始化为0
    bn.weight.data.fill_(1.0)  # 批归一化权重初始化为1


def init_weights(model):  # 根据层类型初始化模型权重的函数
    classname = model.__class__.__name__  # 获取模型类名
    if classname.find("Conv2d") != -1:  # 如果是卷积层
        nn.init.xavier_uniform_(model.weight, gain=np.sqrt(2))  # 使用Xavier初始化权重
        model.bias.data.fill_(0)  # 偏置初始化为0
    elif classname.find("BatchNorm") != -1:  # 如果是批归一化层
        model.weight.data.normal_(1.0, 0.02)  # 权重初始化为均值1.0，标准差0.02的正态分布
        model.bias.data.fill_(0)  # 偏置初始化为0
    elif classname.find("GRU") != -1:  # 如果是GRU层
        for weight in model.parameters():  # 遍历所有参数
            if len(weight.size()) > 1:  # 如果参数维度大于1
                nn.init.orghogonal_(weight.data)  # 使用正交初始化
    elif classname.find("Linear") != -1:  # 如果是线性层
        model.weight.data.normal_(0, 0.01)  # 权重初始化为均值0，标准差0.01的正态分布
        model.bias.data.zero_()  # 偏置初始化为0


def interpolate(x, ratio):  # 时间维度插值函数
    (batch_size, time_steps, classes_num) = x.shape  # 获取输入形状
    upsampled = x[:, :, None, :].repeat(1, 1, ratio, 1)  # 在时间维度重复
    upsampled = upsampled.reshape(batch_size, time_steps * ratio, classes_num)  # 重塑形状
    return upsampled  # 返回上采样结果


def pad_framewise_output(framewise_output, frames_num):  # 帧级输出填充函数
    output = F.interpolate(  # 使用双线性插值
        framewise_output.unsqueeze(1),  # 增加维度
        size=(frames_num, framewise_output.size(2)),  # 设置目标大小
        align_corners=True,  # 对齐角点
        mode="bilinear").squeeze(1)  # 使用双线性插值并压缩维度

    return output  # 返回填充后的输出


class AttBlockV2(nn.Module):  # 注意力块V2定义
    def __init__(self,
                 in_features: int,  # 输入特征维度
                 out_features: int,  # 输出特征维度
                 activation="linear"):  # 激活函数类型
        super().__init__()  # 调用父类初始化

        self.activation = activation  # 设置激活函数
        self.att = nn.Conv1d(  # 注意力卷积层
            in_channels=in_features,  # 输入通道数
            out_channels=out_features,  # 输出通道数
            kernel_size=1,  # 卷积核大小
            stride=1,  # 步长
            padding=0,  # 填充
            bias=True)  # 使用偏置
        self.cla = nn.Conv1d(  # 分类卷积层
            in_channels=in_features,  # 输入通道数
            out_channels=out_features,  # 输出通道数
            kernel_size=1,  # 卷积核大小
            stride=1,  # 步长
            padding=0,  # 填充
            bias=True)  # 使用偏置

        self.init_weights()  # 初始化权重

    def init_weights(self):  # 初始化权重方法
        init_layer(self.att)  # 初始化注意力层
        init_layer(self.cla)  # 初始化分类层

    def forward(self, x):  # 前向传播
        norm_att = torch.softmax(torch.tanh(self.att(x)), dim=-1)  # 计算归一化注意力
        cla = self.nonlinear_transform(self.cla(x))  # 应用非线性变换到分类输出
        x = torch.sum(norm_att * cla, dim=2)  # 计算加权和
        return x, norm_att, cla  # 返回输出、注意力权重和分类输出

    def nonlinear_transform(self, x):  # 非线性变换函数
        if self.activation == 'linear':  # 如果是线性激活
            return x  # 直接返回输入
        elif self.activation == 'sigmoid':  # 如果是sigmoid激活
            return torch.sigmoid(x)  # 应用sigmoid函数


class TimmSED(nn.Module):  # 基于Timm的声音事件检测模型
    def __init__(self, base_model_name: str, pretrained=False, num_classes=24, in_channels=1, n_mels=24):  # 初始化函数
        super().__init__()  # 调用父类初始化

        self.bn0 = nn.BatchNorm2d(n_mels)  # 创建批归一化层

        base_model = timm.create_model(  # 创建基础模型
            base_model_name, pretrained=pretrained, in_chans=in_channels)
        layers = list(base_model.children())[:-2]  # 提取除最后两层外的所有层
        self.encoder = nn.Sequential(*layers)  # 创建编码器

        in_features = base_model.num_features  # 获取特征维度

        self.fc1 = nn.Linear(in_features, in_features, bias=True)  # 创建全连接层
        self.att_block2 = AttBlockV2(  # 创建注意力块
            in_features, num_classes, activation="sigmoid")

        self.init_weight()  # 初始化权重

    def init_weight(self):  # 初始化权重方法
        init_bn(self.bn0)  # 初始化批归一化层
        init_layer(self.fc1)  # 初始化全连接层
        

    def forward(self, input_data):  # 前向传播
        x = input_data.transpose(2,3)  # 转置输入数据维度
        x = torch.cat((x,x,x),1)  # 在通道维度复制三次

        x = x.transpose(2, 3)  # 再次转置维度

        x = self.encoder(x)  # 通过编码器
        
        x = torch.mean(x, dim=2)  # 在频率维度求平均

        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)  # 最大池化
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)  # 平均池化
        x = x1 + x2  # 合并池化结果

        x = x.transpose(1, 2)  # 转置维度
        x = F.relu_(self.fc1(x))  # 应用ReLU激活函数
        x = x.transpose(1, 2)  # 再次转置维度

        (clipwise_output, norm_att, segmentwise_output) = self.att_block2(x)  # 通过注意力块
        logit = torch.sum(norm_att * self.att_block2.cla(x), dim=2)  # 计算logit输出

        output_dict = {  # 创建输出字典
            'logit': logit,  # 保存logit
        }

        return output_dict  # 返回输出字典

In [6]:
base_model_name='eca_nfnet_l0'  # 设置基础模型名称为ECA-NFNet-L0
pretrained=False  # 不使用预训练权重
in_channels=3  # 设置输入通道数为3

MODELS = [f'/kaggle/input/birdclef-2025-sed-models-p/sed{i}.pth' for i in range(3)]  # 创建包含3个模型路径的列表

MODELS  # 显示模型路径列表

['/kaggle/input/birdclef-2025-sed-models-p/sed0.pth',
 '/kaggle/input/birdclef-2025-sed-models-p/sed1.pth',
 '/kaggle/input/birdclef-2025-sed-models-p/sed2.pth']

In [7]:
def prediction(afile):    
    global pred  # 使用全局预测结果字典
    path = test_audio_dir + afile + '.ogg'  # 构建音频文件完整路径
    with torch.inference_mode():  # 使用推理模式，不计算梯度
        sig = audio_to_mel(path)  # 将音频转换为梅尔频谱
        outputs = None  # 初始化输出为None
        for model in models:  # 遍历所有模型
            model.eval()  # 将模型设为评估模式
            p = model(sig)  # 通过模型获取预测结果
            p = torch.sigmoid(p['logit']).detach().cpu().numpy()  # 应用sigmoid，转为numpy数组
            p = adaptive_power(p, top_k=30, min_exp=1.5, max_exp=3.0)  # 应用自适应幂变换
            if outputs is None: outputs = p  # 如果是第一个模型，直接赋值
            else: outputs += p  # 否则累加预测结果
            
        outputs /= len(models)  # 计算模型平均结果
        chunks = [[] for i in range(12)]  # 初始化12个区块的列表
        for i in range(len(chunks)):  # 遍历每个区块      
            chunk_end_time = (i + 1) * 5  # 计算区块结束时间
            row_id = afile + '_' + str(chunk_end_time)  # 构建行ID
            pred['row_id'].append(row_id)  # 添加行ID到结果
            bird_no = 0  # 初始化鸟类编号
            for bird in class_labels:  # 遍历所有鸟类类别       
                pred[bird].append(outputs[i,bird_no])  # 添加该鸟类的预测概率
                bird_no += 1  # 鸟类编号加1
        gc.collect()  # 触发垃圾回收

In [8]:
models = []  # 初始化模型列表
for path in MODELS:  # 遍历所有模型路径
    model = TimmSED(base_model_name=base_model_name,  # 创建TimmSED模型实例
               pretrained=pretrained,  # 设置是否使用预训练
               num_classes=len(class_labels),  # 设置类别数量
               in_channels=in_channels,  # 设置输入通道数
               n_mels=n_mels)  # 设置梅尔频谱频段数
    model.load_state_dict(torch.load(path, weights_only=True, map_location=torch.device('cpu')))  # 加载模型权重
    model.eval()  # 设置模型为评估模式
    models.append(model)  # 将模型添加到列表

In [9]:
pred = {'row_id': []}  # 初始化预测结果字典，包含行ID键
for species_code in class_labels:  # 遍历所有鸟类类别
    pred[species_code] = []  # 为每个鸟类创建空列表
    
start = time.time()  # 记录开始时间
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:  # 创建线程池执行器，最大5个线程
    _ = list(executor.map(prediction, file_list))  # 并行执行预测函数
end_t = time.time()  # 记录结束时间

if debug == True:  # 如果是调试模式
    print(700*(end_t - start)/60/debug_num)  # 打印估计的完整数据集处理时间（分钟）

36.70228918393453


In [10]:
results = pd.DataFrame(pred, columns = ['row_id'] + class_labels)  # 创建结果数据框，包含行ID和所有鸟类类别列

In [ ]:
results.to_csv("submission1.csv", index=False)  # 保存结果为CSV文件，不包含索引列

sub = pd.read_csv('submission1.csv')  # 读取保存的CSV文件
cols = sub.columns[1:]  # 获取除行ID外的所有列名
groups = sub['row_id'].str.rsplit('_', n=1).str[0]  # 从行ID中提取声音文件名
groups = groups.values  # 转换为numpy数组
for group in np.unique(groups):  # 遍历每个唯一的声音文件
    sub_group = sub[group == groups]  # 提取当前声音文件的所有片段
    predictions = sub_group[cols].values  # 获取预测值数组
    new_predictions = predictions.copy()  # 创建预测值副本
    for i in range(1, predictions.shape[0]-1):  # 处理中间片段（应用滑动窗口平滑）
        new_predictions[i] = (predictions[i-1] * 0.2) + (predictions[i] * 0.6) + (predictions[i+1] * 0.2)  # 加权平均
    new_predictions[0] = (predictions[0] * 0.8) + (predictions[1] * 0.2)  # 处理第一个片段
    new_predictions[-1] = (predictions[-1] * 0.8) + (predictions[-2] * 0.2)  # 处理最后一个片段
    sub_group[cols] = new_predictions  # 更新子组的预测值
    sub[group == groups] = sub_group  # 将子组结果放回主数据框
sub.to_csv("submission1.csv", index=False)  # 保存平滑后的结果

<h1 style="color: #6cb4e4;  text-align: center;  padding: 0.25em;  border-top: solid 2.5px #6cb4e4;  border-bottom: solid 2.5px #6cb4e4;  background: -webkit-repeating-linear-gradient(-45deg, #f0f8ff, #f0f8ff 3px,#e9f4ff 3px, #e9f4ff 7px);  background: repeating-linear-gradient(-45deg, #f0f8ff, #f0f8ff 3px,#e9f4ff 3px, #e9f4ff 7px);height:45px;">
<b>
《《《submission2(seresnext)》》》
</b></h1> 

In [12]:
import os  # 导入操作系统接口模块
import gc  # 导入垃圾收集模块
import warnings  # 导入警告控制模块
import logging  # 导入日志模块
import time  # 导入时间相关模块
from pathlib import Path  # 导入路径处理模块
import pandas as pd  # 导入数据处理库
import soundfile as sf  # 导入音频文件读写模块
import torch  # 导入PyTorch深度学习框架
import torch.nn as nn  # 导入神经网络模块
import torch.nn.functional as F  # 导入函数式接口
import timm  # 导入预训练模型库
from glob import glob  # 导入文件路径匹配模块
import torchaudio  # 导入音频处理库
import random  # 导入随机数模块
import itertools  # 导入迭代器工具
import concurrent.futures  # 导入并行处理模块

warnings.filterwarnings("ignore")  # 忽略警告信息
logging.basicConfig(level=logging.ERROR)  # 设置日志级别为ERROR

In [13]:
class CFG:  # 定义配置类
    
    seed = 42  # 随机种子
    print_freq = 100  # 打印频率
    num_workers = 4  # 数据加载器的工作线程数

    stage = 'train_bce'  # 训练阶段标识

    train_datadir = '/kaggle/input/birdclef-2025/train_audio'  # 训练音频目录
    train_csv = '/kaggle/input/birdclef-2025/train.csv'  # 训练CSV文件路径
    test_soundscapes = '/kaggle/input/birdclef-2025/test_soundscapes'  # 测试声音景观目录
    submission_csv = '/kaggle/input/birdclef-2025/sample_submission.csv'  # 提交样例文件
    taxonomy_csv = '/kaggle/input/birdclef-2025/taxonomy.csv'  # 分类学CSV文件路径
    model_files = ['/kaggle/input/bird2025-sed-ckpt/sedmodel.pth'  # 模型文件路径
                  ]
 
    model_name = 'seresnext26t_32x4d'  # 模型名称
    pretrained = False  # 是否使用预训练权重
    in_channels = 1  # 输入通道数

    
    SR = 32000  # 采样率
    target_duration = 5  # 目标音频持续时间（秒）
    train_duration = 10  # 训练音频持续时间（秒）
    
    
    device = 'cpu'  # 计算设备

cfg = CFG()  # 创建配置对象

In [14]:
print(f"Using device: {cfg.device}")  # 打印所使用的计算设备
print(f"Loading taxonomy data...")  # 打印加载分类数据的信息
taxonomy_df = pd.read_csv(cfg.taxonomy_csv)  # 读取鸟类分类数据
species_ids = taxonomy_df['primary_label'].tolist()  # 提取所有鸟类ID列表
num_classes = len(species_ids)  # 计算鸟类类别数量
print(f"Number of classes: {num_classes}")  # 打印类别数量

Using device: cpu
Loading taxonomy data...
Number of classes: 206


In [15]:
def set_seed(seed=42):  # 定义设置随机种子的函数
    """
    设置随机种子以确保结果可重现
    """
    random.seed(seed)  # 设置Python随机模块的种子
    os.environ["PYTHONHASHSEED"] = str(seed)  # 设置Python哈希种子
    np.random.seed(seed)  # 设置NumPy随机种子
    torch.manual_seed(seed)  # 设置PyTorch CPU随机种子
    torch.cuda.manual_seed(seed)  # 设置PyTorch单GPU随机种子
    torch.cuda.manual_seed_all(seed)  # 设置PyTorch多GPU随机种子
    torch.backends.cudnn.deterministic = True  # 使cudnn卷积操作确定性
    torch.backends.cudnn.benchmark = False  # 禁用cudnn基准测试

set_seed(cfg.seed)  # 使用配置中的种子初始化随机性

In [16]:
class AttBlockV2(nn.Module):
    def __init__(self, in_features: int, out_features: int, activation="linear"):
        super().__init__()

        self.activation = activation
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True,
        )
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True,
        )

        self.init_weights()

    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.tanh(self.att(x)), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == "linear":
            return x
        elif self.activation == "sigmoid":
            return torch.sigmoid(x)


def init_layer(layer):
    nn.init.xavier_uniform_(layer.weight)

    if hasattr(layer, "bias"):
        if layer.bias is not None:
            layer.bias.data.fill_(0.0)

def init_bn(bn):
    bn.bias.data.fill_(0.0)
    bn.weight.data.fill_(1.0)

In [17]:
class BirdCLEFModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        
        taxonomy_df = pd.read_csv('/kaggle/input/birdclef-2025/taxonomy.csv')
        self.num_classes = len(taxonomy_df)

        self.bn0 = nn.BatchNorm2d(cfg['n_mels'])
        
        self.backbone = timm.create_model(
            cfg['model_name'],
            pretrained=False,
            in_chans=cfg['in_channels'],
            drop_rate=0.2,
            drop_path_rate=0.2,
        )

        layers = list(self.backbone.children())[:-2]
        self.encoder = nn.Sequential(*layers)
        
        if "efficientnet" in self.cfg['model_name']:
            backbone_out = self.backbone.classifier.in_features
        elif "eca" in self.cfg['model_name']:
            backbone_out = self.backbone.head.fc.in_features
        elif "res" in self.cfg['model_name']:
            backbone_out = self.backbone.fc.in_features
        else:
            backbone_out = self.backbone.num_features
            
        
        self.fc1 = nn.Linear(backbone_out, backbone_out, bias=True)
        self.att_block = AttBlockV2(backbone_out, self.num_classes, activation="sigmoid")

        self.melspec_transform = torchaudio.transforms.MelSpectrogram(
            sample_rate=self.cfg['SR'],
            hop_length=self.cfg['hop_length'],
            n_mels=self.cfg['n_mels'],
            f_min=self.cfg['f_min'],
            f_max=self.cfg['f_max'],
            n_fft=self.cfg['n_fft'],
            pad_mode="constant",
            norm="slaney",
            onesided=True,
            mel_scale="htk",
        )
        if self.cfg['device'] == "cuda":
            self.melspec_transform = self.melspec_transform.cuda()
        else:
            self.melspec_transform = self.melspec_transform.cpu()

        self.db_transform = torchaudio.transforms.AmplitudeToDB(
            stype="power", top_db=80
        )


    def extract_feature(self,x):
        x = x.permute((0, 1, 3, 2))
        frames_num = x.shape[2]
        
        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)
        
        # if self.training:
        #    x = self.spec_augmenter(x)
        
        x = x.transpose(2, 3)
        # (batch_size, channels, freq, frames)
        x = self.encoder(x)
        
        # (batch_size, channels, frames)
        x = torch.mean(x, dim=2)
        
        # channel smoothing
        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2
        
        x = F.dropout(x, p=0.5, training=self.training)
        x = x.transpose(1, 2)
        x = F.relu_(self.fc1(x))
        x = x.transpose(1, 2)
        x = F.dropout(x, p=0.5, training=self.training)
        return x, frames_num
        
    @torch.cuda.amp.autocast(enabled=False)
    def transform_to_spec(self, audio):

        audio = audio.float()
        
        spec = self.melspec_transform(audio)
        spec = self.db_transform(spec)

        if self.cfg['normal'] == 80:
            spec = (spec + 80) / 80
        elif self.cfg['normal'] == 255:
            spec = spec / 255
        else:
            raise NotImplementedError
                
        if self.cfg['in_channels'] == 3:
            spec = image_delta(spec)
        
        return spec

    def forward(self, x):

        with torch.no_grad():
            x = self.transform_to_spec(x)

        x, frames_num = self.extract_feature(x)
        
        (clipwise_output, norm_att, segmentwise_output) = self.att_block(x)
        logit = torch.sum(norm_att * self.att_block.cla(x), dim=2)
        segmentwise_logit = self.att_block.cla(x).transpose(1, 2)
        segmentwise_output = segmentwise_output.transpose(1, 2)

        return torch.logit(clipwise_output)

    def infer(self, x, tta_delta=2):
        with torch.no_grad():
            x = self.transform_to_spec(x)
        x,_ = self.extract_feature(x)
        time_att = torch.tanh(self.att_block.att(x))
        feat_time = x.size(-1)
        start = (
            feat_time / 2 - feat_time * (self.cfg['infer_duration'] / self.cfg['duration_train']) / 2
        )
        end = start + feat_time * (self.cfg['infer_duration'] / self.cfg['duration_train'])
        start = int(start)
        end = int(end)
        pred = self.attention_infer(start,end,x,time_att)

        start_minus = max(0, start-tta_delta)
        end_minus=end-tta_delta
        pred_minus = self.attention_infer(start_minus,end_minus,x,time_att)

        start_plus = start+tta_delta
        end_plus=min(feat_time, end+tta_delta)
        pred_plus = self.attention_infer(start_plus,end_plus,x,time_att)

        pred = 0.5*pred + 0.25*pred_minus + 0.25*pred_plus
        return pred
        
    def attention_infer(self,start,end,x,time_att):
        feat = x[:, :, start:end]
        # att = torch.softmax(time_att[:, :, start:end], dim=-1)
        #             print(feat_time, start, end)
        #             print(att_a.sum(), att.sum(), time_att.shape)
        framewise_pred = torch.sigmoid(self.att_block.cla(feat))
        framewise_pred_max = framewise_pred.max(dim=2)[0]
        # clipwise_output = torch.sum(framewise_pred * att, dim=-1)
        #logits = torch.sum(
        #    self.att_block.cla(feat) * att,
        #    dim=-1,
        #)

        # return clipwise_output
        return framewise_pred_max

In [18]:
def load_sample(path, cfg):
    audio, orig_sr = sf.read(path, dtype="float32")
    seconds = []
    audio_length = cfg.SR * cfg.target_duration
    step = audio_length
    for i in range(audio_length, len(audio) + step, step):
        start = max(0, i - audio_length)
        end = start + audio_length
        if end > len(audio):
            pass
        else:
            seconds.append(int(end/cfg.SR))

    audio = np.concatenate([audio,audio,audio])
    audios = []
    for i,second in enumerate(seconds):
        end_seconds = int(second)
        start_seconds = int(end_seconds - cfg.target_duration)
    
        end_index = int(cfg.SR * (end_seconds + (cfg.train_duration - cfg.target_duration) / 2) ) + len(audio) // 3
        start_index = int(cfg.SR * (start_seconds - (cfg.train_duration - cfg.target_duration) / 2) ) + len(audio) // 3
        end_pad = int(cfg.SR * (cfg.train_duration - cfg.target_duration) / 2) 
        start_pad = int(cfg.SR * (cfg.train_duration - cfg.target_duration) / 2) 
        y = audio[start_index:end_index].astype(np.float32)
        if i==0:
            y[:start_pad] = 0
        elif i==(len(seconds)-1):
            y[-end_pad:] = 0
        audios.append(y)

    return audios

def sigmoid(x):
    s = 1 / (1 + np.exp(-x))
    return s

In [19]:
def find_model_files(cfg):
    """
    Find all .pth model files in the specified model directory
    """
    model_files = []
    
    model_dir = Path(cfg.model_path)
    
    for path in model_dir.glob('**/*.pth'):
        model_files.append(str(path))
    
    return model_files

def load_models(cfg, num_classes):
    """
    Load all found model files and prepare them for ensemble
    """
    models = []
    
    # model_files = find_model_files(cfg)
    model_files = cfg.model_files
    
    if not model_files:
        print(f"Warning: No model files found under {cfg.model_path}!")
        return models
    
    print(f"Found a total of {len(model_files)} model files.")
    
    for i, model_path in enumerate(model_files):
        try:
            print(f"Loading model: {model_path}")
            checkpoint = torch.load(model_path, map_location=torch.device(cfg.device), weights_only=False)
            cfg_temp = checkpoint['cfg']
            cfg_temp['device'] = cfg.device
            
            model = BirdCLEFModel(cfg_temp)
            model.load_state_dict(checkpoint['model_state_dict'])
            model = model.to(cfg.device)
            model.eval()
            model.zero_grad()
            model.half().float()
            
            models.append(model)
        except Exception as e:
            print(f"Error loading model {model_path}: {e}")
    
    return models

def predict_on_spectrogram(audio_path, models, cfg, species_ids):
    """Process a single audio file and predict species presence for each 5-second segment"""
    audio_path = str(audio_path)
    predictions = []
    row_ids = []
    soundscape_id = Path(audio_path).stem

    print(f"Processing {soundscape_id}")
    audio_data = load_sample(audio_path, cfg)
    for segment_idx, audio_input in enumerate(audio_data):
        
        end_time_sec = (segment_idx + 1) * cfg.target_duration
        row_id = f"{soundscape_id}_{end_time_sec}"
        row_ids.append(row_id)
        
        mel_spec = torch.tensor(audio_input, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        mel_spec = mel_spec.to(cfg.device)
        
        if len(models) == 1:
            with torch.no_grad():
                outputs = models[0].infer(mel_spec)
                final_preds = outputs.squeeze()
                # final_preds = torch.sigmoid(outputs).cpu().numpy().squeeze()

        else:
            segment_preds = []
            for model in models:
                with torch.no_grad():
                    outputs = model.infer(mel_spec)
                    probs = outputs.squeeze()
                    # probs = torch.sigmoid(outputs).cpu().numpy().squeeze()
                    segment_preds.append(probs)

            
            final_preds = np.mean(segment_preds, axis=0)
                
        predictions.append(final_preds)

    predictions = np.stack(predictions,axis=0)
    
    return row_ids, predictions

In [20]:
def run_inference(cfg, models, species_ids):
    """Run inference on all test soundscapes"""
    test_files = list(Path(cfg.test_soundscapes).glob('*.ogg'))
    if len(test_files) == 0:
        test_files = sorted(glob(str(Path('/kaggle/input/birdclef-2025/train_soundscapes') / '*.ogg')))[:10]
    
    print(f"Found {len(test_files)} test soundscapes")

    all_row_ids = []
    all_predictions = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        results = list(
        executor.map(
            predict_on_spectrogram,
            test_files,
            itertools.repeat(models),
            itertools.repeat(cfg),
            itertools.repeat(species_ids)
        )
    )

    for rids, preds in results:
        all_row_ids.extend(rids)
        all_predictions.extend(preds)
    
    return all_row_ids, all_predictions

def create_submission(row_ids, predictions, species_ids, cfg):
    """Create submission dataframe"""
    print("Creating submission dataframe...")

    submission_dict = {'row_id': row_ids}
    
    for i, species in enumerate(species_ids):
        submission_dict[species] = [pred[i] for pred in predictions]

    submission_df = pd.DataFrame(submission_dict)

    submission_df.set_index('row_id', inplace=True)

    sample_sub = pd.read_csv(cfg.submission_csv, index_col='row_id')

    missing_cols = set(sample_sub.columns) - set(submission_df.columns)
    if missing_cols:
        print(f"Warning: Missing {len(missing_cols)} species columns in submission")
        for col in missing_cols:
            submission_df[col] = 0.0

    submission_df = submission_df[sample_sub.columns]

    submission_df = submission_df.reset_index()
    
    return submission_df


def smooth_submission(submission_path):
        """
        Post-process the submission CSV by smoothing predictions to enforce temporal consistency.
        
        For each soundscape (grouped by the file name part of 'row_id'), each row's predictions
        are averaged with those of its neighbors using defined weights.
        
        :param submission_path: Path to the submission CSV file.
        """
        print("Smoothing submission predictions...")
        sub = pd.read_csv(submission_path)
        cols = sub.columns[1:]
        groups = sub['row_id'].str.rsplit('_', n=1).str[0].values
        unique_groups = np.unique(groups)
        for group in unique_groups:
            idx = np.where(groups == group)[0]
            sub_group = sub.iloc[idx].copy()
            predictions = sub_group[cols].values
            new_predictions = predictions.copy()
            if predictions.shape[0] > 2:
                new_predictions[0] = (predictions[0]*0.6 + predictions[1]*0.3 + predictions[2]*0.1)
                new_predictions[1] = (predictions[0]*0.2 + predictions[1]*0.6 + predictions[2]*0.2)
                for i in range(2, predictions.shape[0]-2):
                    new_predictions[i] = (predictions[i-2]*0.1 + predictions[i-1]*0.2 + predictions[i]*0.4 + predictions[i+1]*0.2 + predictions[i+2]*0.1)
                new_predictions[-2] = (predictions[-3]*0.2 + predictions[-2]*0.6 + predictions[-1]*0.2)
                new_predictions[-1] = (predictions[-3]*0.1 + predictions[-2]*0.3 + predictions[-1]*0.6)
            sub.iloc[idx, 1:] = new_predictions
        sub.to_csv(submission_path, index=False)
        print(f"Smoothed submission saved to {submission_path}")
    



In [21]:
def main():  # 定义主函数
    start_time = time.time()  # 记录开始时间
    print("Starting BirdCLEF-2025 inference...")  # 打印开始推理的信息

    models = load_models(cfg, num_classes)  # 加载模型
    
    if not models:  # 如果没有找到模型
        print("No models found! Please check model paths.")  # 打印错误信息
        return  # 提前返回
    
    print(f"Model usage: {'Single model' if len(models) == 1 else f'Ensemble of {len(models)} models'}")  # 打印模型使用情况

    row_ids, predictions = run_inference(cfg, models, species_ids)  # 运行推理获取结果

    submission_df = create_submission(row_ids, predictions, species_ids, cfg)  # 创建提交数据框

    submission_path = 'submission.csv'  # 设置提交文件路径
    submission_df.to_csv(submission_path, index=False)  # 保存提交文件
    print(f"Submission saved to {submission_path}")  # 打印保存信息

    smooth_submission(submission_path)  # 对提交结果进行平滑处理
    
    end_time = time.time()  # 记录结束时间
    print(f"Inference completed in {(end_time - start_time)/60:.2f} minutes")  # 打印完成时间

In [22]:
if __name__ == "__main__":  # 如果作为主程序运行
    main()  # 执行主函数

Starting BirdCLEF-2025 inference...
Found a total of 1 model files.
Loading model: /kaggle/input/bird2025-sed-ckpt/sedmodel.pth
Model usage: Single model
Found 10 test soundscapes
Processing H02_20230420_074000
Processing H02_20230420_112000
Processing H02_20230420_154500
Processing H02_20230420_164000
Processing H02_20230420_223500
Processing H02_20230421_093000
Processing H02_20230421_113500
Processing H02_20230421_170000
Processing H02_20230421_190500
Processing H02_20230421_233500
Creating submission dataframe...
Submission saved to submission.csv
Smoothing submission predictions...
Smoothed submission saved to submission.csv
Inference completed in 0.59 minutes


<h1 style="color: #6cb4e4;  text-align: center;  padding: 0.25em;  border-top: solid 2.5px #6cb4e4;  border-bottom: solid 2.5px #6cb4e4;  background: -webkit-repeating-linear-gradient(-45deg, #f0f8ff, #f0f8ff 3px,#e9f4ff 3px, #e9f4ff 7px);  background: repeating-linear-gradient(-45deg, #f0f8ff, #f0f8ff 3px,#e9f4ff 3px, #e9f4ff 7px);height:45px;">
<b>
《《《最终融合》》》
</b></h1> 

In [23]:
# ------------------------------------------- #
# [重要]
# * 融合权重
# ------------------------------------------- #
# 权重设置 sub_w=[0.75, 0.25]

In [24]:
# list_TARGETs = sorted(os.listdir('/kaggle/input/birdclef-2025/train_audio/'))
# list_targets_0 = [f'{TARGET} 0' for TARGET in list_TARGETs]
# list_targets_1 = [f'{TARGET} 1' for TARGET in list_TARGETs]

# df0 = pd.read_csv("/kaggle/working/submission.csv")
# df1 = pd.read_csv("/kaggle/working/submission1.csv")

# df0 = df0.rename(columns={TARGET : f'{TARGET} 0' for TARGET in list_TARGETs})
# df1 = df1.rename(columns={TARGET : f'{TARGET} 1' for TARGET in list_TARGETs})

# dfs = pd.merge(df0,df1,on=['row_id'])

# for i in range(len(list_TARGETs)):
#     dfs[list_TARGETs[i]] = dfs[list_targets_0[i]]*sub_w[0] + sub_w[1]*dfs[list_targets_1[i]]
             
# for col0,col1 in zip(list_targets_0, list_targets_1):
#     del dfs[col0]
#     del dfs[col1]
    
    
# dfs.to_csv("submission.csv", index=False)

In [25]:
from scipy.special import logit, expit  # 导入特殊函数logit和expit（逆logit）

list_TARGETs = sorted(os.listdir('/kaggle/input/birdclef-2025/train_audio/'))  # 获取并排序所有鸟类标签

df0 = pd.read_csv("/kaggle/working/submission.csv")  # 读取第一个模型的预测结果
df1 = pd.read_csv("/kaggle/working/submission1.csv")  # 读取第二个模型的预测结果

# 保证列名一致
for TARGET in list_TARGETs:  # 遍历所有鸟类标签
    if f'{TARGET} 0' in df0.columns:  # 如果列名带有后缀
        df0[TARGET] = df0[f'{TARGET} 0']  # 规范化列名
    if f'{TARGET} 1' in df1.columns:  # 如果列名带有后缀
        df1[TARGET] = df1[f'{TARGET} 1']  # 规范化列名

cols = [col for col in df0.columns if col != 'row_id']  # 获取除行ID外的所有列名

blend = df0[['row_id']].copy()  # 创建仅含行ID的新数据框
for col in cols:  # 遍历所有鸟类列
    # 在对数空间中加权融合，防止极端概率导致logit溢出
    blend[col] = expit(0.75*logit(df0[col].clip(1e-6, 1-1e-6)) + 0.25*logit(df1[col].clip(1e-6, 1-1e-6)))
blend.to_csv("submission.csv", index=False)  # 保存融合后的结果文件

In [26]:
import numpy as np  # 导入数值计算库
import pandas as pd  # 导入数据处理库

# 鸟类共现调整
num_classes = len(list_TARGETs)  # 获取鸟类类别数量
co_matrix = np.eye(num_classes)  # 创建单位矩阵作为共现矩阵（可以用实际统计的共现数据替换）

def co_occurrence_boost(sub_path, co_matrix, threshold=0.7):  # 定义基于共现的预测增强函数
    sub = pd.read_csv(sub_path)  # 读取提交文件
    cols = [col for col in sub.columns if col != 'row_id']  # 获取所有鸟类列名
    preds = sub[cols].values  # 提取预测值数组
    boosted = preds.copy()  # 创建预测值的副本
    for i in range(preds.shape[1]):  # 遍历每个鸟类类别
        high_idx = preds[:, i] > threshold  # 找出高置信度预测
        for j in range(preds.shape[1]):  # 遍历每个鸟类类别
            if i != j:  # 如果不是同一个类别
                boosted[high_idx, j] += preds[high_idx, i] * co_matrix[i, j]  # 基于共现矩阵增强预测
    boosted = np.clip(boosted, 0, 1)  # 将值限制在0到1之间
    sub[cols] = boosted  # 更新数据框中的预测值
    sub.to_csv(sub_path, index=False)  # 保存更新后的预测结果

co_occurrence_boost("submission.csv", co_matrix, threshold=0.7)  # 应用共现增强到最终结果